In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreadscleaned/df.csv


In [2]:
df = pd.read_csv('/kaggle/input/goodreadscleaned/df.csv')

## Cluster the books based on the description. 

In [3]:
#importing basic python libraries
import matplotlib.pyplot as plt
import seaborn as sns

# text preprocessing
import regex
import nltk

#word embedding
import gensim

#clustering
import sklearn

#connection to data scoursce in Hana cloud
#import hana_ml
    

In [4]:
import re, string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#import contractions
#from sklearn.feature_extraction.text import TfidVectorizer

In [5]:
df.head(5)

,book_id,author_id,authors,title,description,publisher,genres,avg_rating,ratings_count,num_pages,pub_year,similar_books,url
0,2767052,153394,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,Scholastic Press,"['favorites', 'currently-reading', 'to-read', ...",4.34,4899965,374,2008.0,"['1902241', '146499', '954674', '9917938', '10...",https://www.goodreads.com/book/show/2767052-th...
1,3,1077326,J.K. Rowling,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,Scholastic Inc,"['to-read', 'favorites', 'fantasy', 'young-adu...",4.45,4765497,320,1997.0,"['13830', '127586', '121822', '37586', '616435...",https://www.goodreads.com/book/show/3.Harry_Po...
2,2657,1825,Harper Lee,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,Harper Perennial Modern Classics,"['to-read', 'favorites', 'classics', 'classic'...",4.26,3255518,324,2006.0,"['1934', '2156', '15638', '53835', '77142', '5...",https://www.goodreads.com/book/show/2657.To_Ki...
3,4671,3190,F. Scott Fitzgerald,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",Scribner,"['to-read', 'classics', 'favorites', 'fiction'...",3.89,2758812,180,2004.0,"['48203', '337113', '176972', '188087', '10956...",https://www.goodreads.com/book/show/4671.The_G...
4,11870085,1406384,John Green,The Fault in Our Stars,"There is an alternate cover edition .\n""I fel...",Dutton Books,"['to-read', 'favorites', 'young-adult', 'ficti...",4.26,2429317,313,2012.0,"['10051706', '11418182', '10327303', '9627755'...",https://www.goodreads.com/book/show/11870085-t...


In [6]:
df = df.drop('url', axis =1)

# Cleaning Description Column

In [7]:
books = df[['book_id','title','description']]
books.head()

,book_id,title,description
0,2767052,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...
2,2657,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...
3,4671,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ..."
4,11870085,The Fault in Our Stars,"There is an alternate cover edition .\n""I fel..."


## 1. Text Preprocessing

**Tokenization**: The text in the descriotion column must be parsed and transform into a list of words.

**Remove special characters and punctuation**

**Remove "stop words"**: Commonly used words without any specific connocation (e.g., "the","a", "an", "in")

**Use Natural Language Toolkit (NLTK) and Regular Expressions (RegEx)** to clean up and tokenize our text.



In [8]:
#Dropping the missing values
books = books.dropna(axis = 0 , subset = ["description"])

In [9]:
# Prepare the book description column 
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stopwords=set(stopwords.words("english")) 
import regex as re


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Transform to the lowercase
books['tokens'] = books['description'].apply(lambda x: x.lower())

In [26]:
books['tokens']

0         winning make famous losing means certain death...
1         harry potter life miserable parents dead stuck...
2         unforgettable novel childhood sleepy southern ...
3         great gatsby scott fitzgerald third book stand...
4         alternate cover edition "i fell love way fall ...
                                ...                        
869626    system solve problem harry leonnoff fix power ...
869627    ruth hard life see way sees charmed happy howe...
869628    examines history culture changing fortunes len...
869629    july 1937 beloved amelia lost mid pacific crea...
869630    cambridge scholar nathan tobin discovers ancie...
Name: tokens, Length: 826846, dtype: object

In [11]:
#Remove punctuations
books['tokens']  = books['tokens'].map(lambda x: re.sub("[-,\.!?;\'\(\)]", ' ',x))

In [12]:
#Remove stopwords
books['tokens']  = books['tokens'].apply(lambda x: ' '.join([t for t in x.split() if not t in stopwords]))

In [13]:
#Remove short tokens
books['tokens'] = books['tokens'].apply(lambda x:' '.join([t for t in x.split()if len(t)>1]))

In [14]:
#Remove extra spaces
books['tokens'] = books['tokens'].map(lambda x: re.sub(' +', ' ', x))

In [15]:
#Remove duplicate tokens
books['tokens'] = books['tokens'].apply(lambda x: ' '.join(list(dict.fromkeys(x.split()))))

In [16]:
#Drop the duplicates
books = books.drop_duplicates('tokens')

In [17]:
# Remove any non-alphanumeric characters
#books['tokens']=books['tokens'].apply(lambda x: ' '.join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [18]:
books[['book_id', 'title', 'description','tokens']]

,book_id,title,description,tokens
0,2767052,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,winning make famous losing means certain death...
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,harry potter life miserable parents dead stuck...
2,2657,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,unforgettable novel childhood sleepy southern ...
3,4671,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",great gatsby scott fitzgerald third book stand...
4,11870085,The Fault in Our Stars,"There is an alternate cover edition .\n""I fel...","alternate cover edition ""i fell love way fall ..."
...,...,...,...,...
869626,2314733,Fixer,"If the system won't solve your problem, Harry ...",system solve problem harry leonnoff fix power ...
869627,26733966,Wake Up,"Ruth has had a hard life, but she doesn't see ...",ruth hard life see way sees charmed happy howe...
869628,1040719,The Lenapes (Paperback),"Examines the history, culture, and changing fo...",examines history culture changing fortunes len...
869629,15775045,Amelia Earhart--Case Closed!,"On July 2, 1937 our beloved Amelia was lost ov...",july 1937 beloved amelia lost mid pacific crea...


Rescources on techniques for pre-processing:
* https://tutorialspoint.dev/language/python/removing-stop-words-nltk-python 
* https://en.wikipedia.org/wiki/Regular_expression

## Word Embedding

**Use Global Vectors for Word Representation (GloVe):** GloVe is a word2vec model that is to say an unsupervised learning algorithm for obtaining vector representations for words.It allows you to take any corpus of text and transform each word into a position in high-dimentional space. 
    
**Download a pretrained model** that was trained on a broder corpus:Wikipidia
Each line of text file contains a word follow by N number. The N number describe the vector of the word position. N is varid depending on which model you choose to download. 
    
    
**More Resources:** 
* https://faculty.ai/tech-blog/glove/
* https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db 


In [19]:
import gensim.downloader as gensim_api

In [20]:
model = gensim_api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [31]:
books['tokens'] = books['tokens'].astype(str)

In [43]:
import numpy as np
from gensim.models import Word2Vec


# generate embedded features for each book
features = []
for tokens in books['tokens']:
    token_features = []
    for token in tokens:
        try:
            token_features.append(model[token])
        except KeyError:
            continue
    features.append(np.mean(token_features, axis=0))

# add features to DataFrame
for i in range(100):
    feature = 'f_' + str(i)
    books[feature] = [f[i] for f in features]

# define embedding column names
embedding = ['f_' + str(i) for i in range(100)]


IndexError: invalid index to scalar variable.

In [44]:
# create an empty array to hold the feature vectors
features = np.zeros((len(books), 100))

# loop over each book description and compute the average embedding
for i, tokens in enumerate(books['tokens']):
    token_list = tokens.split()
    embeddings = []
    for token in token_list:
        if token in model:
            embeddings.append(model[token])
    if len(embeddings) > 0:
        avg_embedding = np.mean(embeddings, axis=0)
    else:
        avg_embedding = np.zeros(100)
    features[i] = avg_embedding

# add the features to the dataframe
for i in range(100):
    feature = 'f_' + str(i)
    books[feature] = features[:, i]

# create a list of the embedding feature names
embedding = ['f_' + str(i) for i in range(100)]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [32]:
#Now that we downloaded tour word2vec model, we can apply it to every book description to embed the book into a multidimentional space.
features=[]
for i,book in books.iterrows():
    tokens_features=[]
    for word in book['tokens'].split():
        try:
            tokens_features.append(model[word])
        except:
            continue
    features.append(np.mean(np.array(tokens_features),axis=0))
    


In [39]:
for i in range(100):
    feature='f_'+str(i)
    books[feature]=[f[i] for f in features]
embedding=['f_'+str(i) for i in range(100)]

IndexError: invalid index to scalar variable.

In [34]:
embedding=['f_'+str(i) for i in range(100)]

In [33]:
for i in range(100):
    feature='f_'+str(i)
    books[feature]=[f[i] for f in features]
    
    
#del features
#embedding=['f_'+str(i) for i in range(100)]

IndexError: invalid index to scalar variable.

In [ ]:
#As a resulting of our word embedding technique we associated each book to a 100 -dim numerical vector:
books[['title']+embedding]

In [30]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826846 entries, 0 to 869630
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   book_id      826846 non-null  int64 
 1   title        826846 non-null  object
 2   description  826846 non-null  object
 3   tokens       826846 non-null  object
dtypes: int64(1), object(3)
memory usage: 31.5+ MB


# KMeans Clustering

Now that we have represented each book by a point in a multidimentional space, we can use the distance between each books to find out which books are similar to each other. 

**Using Cluster Analysis**: The similar books will be grouped togther.

**Setting the number of group to 10**: We use KMeans clustering algorithm in the form og MiniBatchKmeans

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
n_clus =10
km = MiniBatchKMeans(n_clusters = n_clus, bach_size = 50, random_state = 42, max_iter = 1000)
y_kmeans = km.fit_predict(books[embedding])
books['kmeans_cluster']= y_kmeans

## Cluster Profiling

Examine the most representative book from each cluster to see if the clustering work effectively.

In [ ]:
for cluster in range(n_clus):
    print('************* ')
    print('- CLUSTER ',str(cluster))
    print('*************')
    bks=books[books['kmeans_cluster']==cluster]
    most_representative_docs = np.argsort(
    np.linalg.norm(bks[embedding] - km.cluster_centers_[cluster], axis=1)
)
    del bks
    centroid_index= most_representative_docs[0]
    centroid=[]
    for i in range(100):
        feature='f_'+str(i)
    for d in most_representative_docs[:10]:
        print(books.reset_index().Book_Description[d])
        print("--")

We can notice a certain homogeneity between the descriptions belongingto the same 